In [2]:
import pandas as pd
import numpy as np

### Compile all data

In [3]:
data = map(pd.read_csv, [   'data\PFW_1988_1995_public.csv', 
                            'data\PFW_1996_2000_public.csv',
                            'data\PFW_2001_2005_public.csv',
                            'data\PFW_2006_2010_public.csv',
                            'data\PFW_2011_2015_public.csv',
                            'data\PFW_2016_2020_public.csv',
                            'data\PFW_2021_public.csv'])
df = pd.concat(data, ignore_index=True)
df.head(1)

,LOC_ID,LATITUDE,LONGITUDE,SUBNATIONAL1_CODE,ENTRY_TECHNIQUE,SUB_ID,OBS_ID,Month,Day,Year,...,species_code,how_many,valid,reviewed,day1_am,day1_pm,day2_am,day2_pm,effort_hrs_atleast,snow_dep_atleast
0,L20416,35.02262,-93.47239,US-AR,POSTCODE LAT/LONG LOOKUP,S338425,OBS4297367,12,17,1994,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Note size of dataset

In [4]:
df.shape

(36971663, 40)

In [5]:
df['HOW_MANY'].sum() + df['how_many'].sum()

179099099.0

### Replace nulls in duplicate columns

In [6]:
df.loc[df['LATITUDE'].isna(), 'LATITUDE'] =  df['latitude'] 
df.loc[df['LONGITUDE'].isna(), 'LONGITUDE'] =  df['longitude']
df.loc[df['SPECIES_CODE'].isna(), 'SPECIES_CODE'] =  df['species_code']
df.loc[df['HOW_MANY'].isna(), 'HOW_MANY'] =  df['how_many']    


### Set all days to 1 and create date column so data can be grouped by year and month

In [7]:
df['Day'] = 1
df['DATE'] = pd.to_datetime(df[['Year','Month', 'Day']])

### Isolate relevant columns

In [8]:

df = df[['DATE','SPECIES_CODE','LATITUDE','LONGITUDE','HOW_MANY']]

### Drop data with null values

In [9]:
df.isna().sum()


DATE              0
SPECIES_CODE      0
LATITUDE        254
LONGITUDE       254
HOW_MANY          0
dtype: int64

In [10]:
df = df.dropna()

### Round lattitude and longitude to make grouping easier

In [11]:
df['LATITUDE'] = df['LATITUDE'].round(0)
df['LONGITUDE'] = df['LONGITUDE'].round(0)

### Get common names from dictionary

In [47]:
species_keys = pd.read_excel('data/metadata/FeederWatch_Data_Dictionary.xlsx',sheet_name="Species Codes", skiprows=1)
species_keys = species_keys[['REPORT_AS','PRIMARY_COM_NAME']]

### Remove variations

In [48]:
species_keys['PRIMARY_COM_NAME'] = species_keys['PRIMARY_COM_NAME'].str.replace(r"\(.*\)","",regex=True)
species_keys['PRIMARY_COM_NAME'] = species_keys['PRIMARY_COM_NAME'].str.strip()
species_keys.drop_duplicates(inplace=True)

### Merge common names with dataframe

In [54]:
df = pd.merge(df,species_keys, left_on='SPECIES_CODE', right_on='REPORT_AS', how='left')

### Remove key columns

In [55]:
df.drop(['SPECIES_CODE','REPORT_AS'], axis=1, inplace=True)

### Group birds counted by day and location

In [57]:
grouped_df = df.groupby(['DATE','PRIMARY_COM_NAME','LATITUDE','LONGITUDE']).sum()

In [58]:
grouped_df['HOW_MANY'].sum()

177178141.0

In [60]:
grouped_df.shape

(3271194, 1)

In [61]:
grouped_df.to_csv('bird_data.csv')